In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from gensim.models.doc2vec import Doc2Vec

caminho_modelo = 'modelos/doc2vec.text_0_100_10'

In [3]:
# Carregamento do dataset.
df = pd.read_csv('../dados/articles_limpo.csv')
df.columns

Index(['title', 'text', 'date', 'category', 'link'], dtype='object')

In [4]:
# Carregando o modelo doc2vec (títulos).
modelo = Doc2Vec.load(caminho_modelo)

In [5]:
from classificadores import Classificador

c =  Classificador()

caminho_df = '../dados/articles_limpo.csv'
df = pd.read_csv(caminho_df)
classif, s_classif = c.pre_proc(df)
#treino, teste = c.dataset_treino(classif, s_classif, modelo)
c.dataset_treino(classif, s_classif, modelo)
clf = LogisticRegression(random_state=0, max_iter=150, solver='sag')#, C=0.8)


In [18]:
# Treinando o modelo.
"""
vetores = list()
classes = list()
for index, row in treino.iterrows():
    vetores.append(row['vetor'])
    classes.append(row['classe'])
"""

#clf.fit(vetores, classes)
clf.fit(list(classif['vetor']), list(classif['classe']))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=150,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
# Classificando as novas amostras.
vetores = list()
for index, row in s_classif.head().iterrows():
    vetores.append(row['vetor'])
    print(clf.predict([row['vetor']]))
#classificados = clf.predict(vetores)

['poder']
['mercado']
['mundo']
['poder']
['poder']


In [22]:
list(classif['vetor'])

[array([-0.68763316,  0.296071  , -0.29345122,  0.13909602,  0.2200898 ,
        -0.37040627,  0.4277301 ,  0.2996643 ,  0.35402507,  0.20041254,
        -0.44556764, -0.64214027, -0.12922114,  0.6504648 , -0.04470087,
        -0.12994397, -0.23151459, -0.3640144 , -0.40508842,  0.09780636,
        -0.35521302,  0.09203805, -0.32734448,  0.07916676,  0.9831191 ,
         0.4907808 ,  0.39729026, -0.55566263,  0.2520517 , -0.21628866,
         0.07056524, -0.12281249,  0.11053107, -0.08912621, -0.21644741,
        -0.78776973,  0.07769784, -0.21876097,  0.28169328, -0.3319423 ,
         0.45098725,  0.12730908, -1.1354214 ,  0.00643594,  0.32662007,
         0.8070355 , -0.49291074,  0.22924165,  0.584385  ,  0.40847835,
        -0.17591614,  0.16080558,  0.20791088,  0.13424583,  0.16275351,
         0.1580408 ,  0.06158572, -0.02840583, -0.08020692, -0.5146887 ,
         0.4180439 ,  0.10902718, -0.19080582,  0.15071954,  0.5080302 ,
        -0.4350495 , -0.08991918, -0.02563659,  0.3

In [181]:
# Treinando o Naive Bayes.
gnb = GaussianNB()
naive = gnb.fit(vetores_treino[:69854], classes_treino[:69854])
# 29937
#for vetor, classe in zip(vetores_treino[29937:], classes_treino[29937:]):
#    naive.predict([vetor, classe])
acc = naive.score(vetores_treino[29937:], classes_treino[29937:])
acc

0.5900508195547921

In [192]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(vetores_treino[:69854], classes_treino[:69854])
clf.score(vetores_treino[29000:], classes_treino[29000:])

/home/welton/mineracao/StoryTracker/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8339129043053151